In [2]:
!sudo apt-get install -y libcrypt-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcrypt-dev is already the newest version (1:4.4.27-1).
libcrypt-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [3]:
%%writefile EncryptSHA512.c

#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <unistd.h>
/******************************************************************************
  This program is used to set challenges for password cracking programs.
  Encrypts using SHA-512.

  Compile with:
    cc -o EncryptSHA512 EncryptSHA512.c -lcrypt

  To encrypt the password "pass":
    ./EncryptSHA512 pass

  It doesn't do any checking, just does the job or fails ungracefully.

  Dr Kevan Buckley, University of Wolverhampton, 2017. Modified by Dr. Ali Safaa 2019
******************************************************************************/

#define SALT "$6$AS$"

int main(int argc, char *argv[]){

  printf("%s\n", crypt(argv[1], SALT));

  return 0;
}


Writing EncryptSHA512.c


In [4]:
!gcc -o EncryptSHA512 EncryptSHA512.c -lcrypt

In [5]:
!./EncryptSHA512 ZZ99

$6$AS$P.Wy8B/NjpVgwGDKZ1uafxVzLNC7UpfX4yBca4BB03TvxHd0hRhjo0.qr1SpHDU2tzOTwTaVB5/8wm8f6Wgcf.


In [6]:
%%writefile password_thead.c
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <pthread.h>
#include <string.h>
#include <stdatomic.h>

#define SALT "$6$AS$"
#define THREAD_COUNT 4  // Number of worker threads

atomic_int found = 0;

struct ThreadArgs {
    int thread_id;
    char *encryptedPassword;
    char foundPassword[5];  // Enough space to store the found password
};

char* encryptPassword(char* password, struct crypt_data* cdata) {
    return crypt_r(password, SALT, cdata);
}

void* crackPassword(void* arguments) {
    struct ThreadArgs* args = (struct ThreadArgs*)arguments;
    char letter1, letter2, number1, number2;
    char password[5], *encryptedGuess;
    struct crypt_data cdata;
    cdata.initialized = 0; // To initialize the crypt_data structure

    // Total combinations of letter1 and letter2 (26 * 26)
    int totalLetterCombinations = 26 * 26;
    // Calculate the range of letter combinations each thread should work on
    int range = totalLetterCombinations / THREAD_COUNT;
    // Calculate the start index for this thread
    int start = args->thread_id * range;
    // Calculate the end index for this thread
    int end = start + range;

    for (int i = start; i < end && !found; i++) {
        letter1 = 'A' + (i / 26);
        letter2 = 'A' + (i % 26);
        for (number1 = '0'; number1 <= '9' && !found; number1++) {
            for (number2 = '0'; number2 <= '9' && !found; number2++) {
                password[0] = letter1;
                password[1] = letter2;
                password[2] = number1;
                password[3] = number2;
                password[4] = '\0';

                encryptedGuess = encryptPassword(password, &cdata);
                if (strcmp(args->encryptedPassword, encryptedGuess) == 0) {
                    strncpy(args->foundPassword, password, 5);
                    atomic_store(&found, 1);
                    printf("Password found: %s\n", args->foundPassword);
                    return NULL;
                }
            }
        }
    }
    return NULL;
}

int main() {
    char encryptedPassword[100];
    printf("Enter the encrypted password: ");
    if (scanf("%99s", encryptedPassword) != 1) {
        fprintf(stderr, "Failed to read encrypted password.\n");
        return EXIT_FAILURE;
    }

    if (strlen(encryptedPassword) < 13) {
        fprintf(stderr, "Invalid encrypted password format.\n");
        return EXIT_FAILURE;
    }

    pthread_t threads[THREAD_COUNT];
    struct ThreadArgs threadArgs[THREAD_COUNT];

    for (int i = 0; i < THREAD_COUNT; i++) {
        threadArgs[i].thread_id = i;
        threadArgs[i].encryptedPassword = encryptedPassword;
        if (pthread_create(&threads[i], NULL, crackPassword, (void*)&threadArgs[i]) != 0) {
            fprintf(stderr, "Failed to create thread.\n");
            return EXIT_FAILURE;
        }
    }

    for (int i = 0; i < THREAD_COUNT; i++) {
        if (pthread_join(threads[i], NULL) != 0) {
            fprintf(stderr, "Failed to join thread.\n");
            return EXIT_FAILURE;
        }
    }

    if (!found) {
        printf("Password not found.\n");
    }

    return 0;
}


Writing password_thead.c


In [7]:
!gcc -o password_thead password_thead.c -lcrypt -lpthread

In [8]:
!./password_thead

Enter the encrypted password: $6$AS$P.Wy8B/NjpVgwGDKZ1uafxVzLNC7UpfX4yBca4BB03TvxHd0hRhjo0.qr1SpHDU2tzOTwTaVB5/8wm8f6Wgcf.
Password found: ZZ99
